<a href="https://colab.research.google.com/github/Abhishek-TyRnT/TyRnT/blob/master/Malaria_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link="https://drive.google.com/open?id=1FoZ5XNPxsKB75bq7zYZ8rpH5PJoRB988"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('cell_images.zip')

In [0]:
from zipfile import ZipFile
with ZipFile("cell_images.zip",'r') as zip:
  zip.extractall()

In [0]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
data_dir="cell_images"
category_1="Parasitized"
category_2="Uninfected"
path_1=os.path.join(data_dir,category_1)
path_2=os.path.join(data_dir,category_2)
def image_generator(start,stop):
 i=start
 for j in range(start,stop):
   image_1=cv2.imread(os.path.join(path_1,os.listdir(path_1)[j]),cv2.IMREAD_COLOR)
   image_1=tf.image.resize(images=image_1,size=[224,224])
   image_1=tf.reshape(image_1,shape=(1,224,224,3))
   image_2=cv2.imread(os.path.join(path_2,os.listdir(path_2)[j]),cv2.IMREAD_COLOR)
   image_2=tf.image.resize(images=image_2,size=[224,224])
   image_2=tf.reshape(image_2,shape=(1,224,224,3))
   if(start<=i and i<stop):
    yield image_1/255,[True,False]
    yield image_2/255,[False,True]
    i+=1

In [0]:
import tensorflow as tf
dataset_train=tf.data.Dataset.from_generator(image_generator,args=[0,13780//2],output_types=(tf.float32,tf.int32),output_shapes=(tf.TensorShape([None,224,224,3]),tf.TensorShape([2])))
dataset_val=tf.data.Dataset.from_generator(image_generator,args=[(13780//2)+1,13780],output_types=(tf.float32,tf.int32),output_shapes=(tf.TensorShape([None,224,224,3]),tf.TensorShape([2])))

In [0]:
dataset_train=dataset_train.shuffle(buffer_size=100)
dataset_val=dataset_val.shuffle(buffer_size=100)

In [0]:
dataset_train=dataset_train.batch(batch_size=10)
dataset_val=dataset_val.batch(batch_size=100)

In [0]:
i=0
for img in dataset_train.take(150):
  if i>137:
   plt.imshow(img[0][0])
   plt.show()
  print(i)
  i+=1

Alex net

In [48]:
from tensorflow.keras import Model
inp=tf.keras.Input(shape=(224,224,3),dtype=tf.float32)
network=tf.keras.layers.Conv2D(filters=96,kernel_size=(11,11),strides=4,padding='SAME',activation='relu')(inp)
network=tf.nn.local_response_normalization(network,depth_radius=2,alpha=0.00002,beta=0.75)
network=tf.keras.layers.Conv2D(filters=96,kernel_size=(3,3),strides=2,padding='VALID',activation=None)(network)
network=tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2)(network)
network=tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=1,padding='SAME',activation='relu')(network)
network=tf.nn.local_response_normalization(network,depth_radius=2,alpha=0.00002,beta=0.75)
network=tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2)(network)
network=tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=1,padding='SAME',activation='relu')(network)
network=tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=1,padding='SAME',activation='relu')(network)
network=tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=1,padding='SAME',activation='relu')(network)
network=tf.keras.layers.Flatten()(network)
network=tf.keras.layers.Dense(units=4096,activation='relu')(network)
network=tf.keras.layers.Dropout(rate=0.5)(network)
network=tf.keras.layers.Dense(units=1024,activation='relu')(network)
network=tf.keras.layers.Dropout(rate=0.5)(network)
output=tf.keras.layers.Dense(units=2,activation='softmax')(network)

model=Model(inputs=inp,outputs=output)
tf.keras.utils.plot_model(model)
model.compile(optimizer="adam",loss=tf.losses.BinaryCrossentropy(),metrics=["accuracy"])
model.fit_generator(dataset_train,epochs=10,validation_data=dataset_val,steps_per_epoch=1370,validation_steps=1370)

Train for 1370 steps, validate for 1370 steps
Epoch 1/10
1370/1370 [==============================] - 301s 219ms/step - loss: 0.6946 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
1370/1370 [==============================] - 297s 217ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
1370/1370 [==============================] - 298s 217ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
1370/1370 [==============================] - 297s 217ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
1370/1370 [==============================] - 298s 217ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/10
1370/1370 [==============================] - 297s 217ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/10
1370/1370 [==============================] - 297s 217ms

In [50]:
for img in dataset_val.take(10):
  y=model.predict(img[0])
  print(y,img[1])

[[1. 0.]] tf.Tensor([0 1], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([0 1], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([0 1], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([1 0], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([0 1], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([1 0], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([1 0], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([1 0], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([1 0], shape=(2,), dtype=int32)
[[1. 0.]] tf.Tensor([1 0], shape=(2,), dtype=int32)


In [5]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 421.8MB 34kB/s 
     |████████████████████████████████| 450kB 42.2MB/s 
     |████████████████████████████████| 3.8MB 34.0MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall